In [ ]:
import pandas as pd

In [2]:
import pandas as pd

# create a small CSV with mixed / messy values and read it back to simulate importing a "mixed" dataset
sample = {
	'id': [1, 2, 3, 4, 5, 6],
	'age': ['25', 'thirty', '40', 'NaN', '', '35'],
	'salary': ['1000', '$2,500', '3000.0', 'N/A', '0', '$1,200'],
	'joined': ['2020-01-15', '15/02/2021', '2021-03-01', 'unknown', '', '2020-12-31'],
	'active': ['yes', 'no', 1, 0, 'True', 'FALSE'],
	'score': ['85.5', '90', '—', '70.0', 'N/A', '88']
}
df = pd.DataFrame(sample)
df.to_csv('mixed_dataset.csv', index=False)

# import the dataset
df_raw = pd.read_csv('mixed_dataset.csv')
print("Raw data:")
print(df_raw)
print("\nDtypes (before cleaning):")
print(df_raw.dtypes)

Raw data:
   id     age  salary      joined active score
0   1      25    1000  2020-01-15    yes  85.5
1   2  thirty  $2,500  15/02/2021     no    90
2   3      40  3000.0  2021-03-01      1     —
3   4     NaN     NaN     unknown      0  70.0
4   5     NaN       0         NaN   True   NaN
5   6      35  $1,200  2020-12-31  FALSE    88

Dtypes (before cleaning):
id         int64
age       object
salary    object
joined    object
active    object
score     object
dtype: object


In [5]:
df_raw.isnull().sum()

id        0
age       2
salary    1
joined    1
active    0
score     1
dtype: int64

In [8]:
df_raw['age'].value_counts()

age
25        1
thirty    1
40        1
35        1
Name: count, dtype: int64

In [9]:
df_raw['age'].isnull().sum()

np.int64(2)

In [12]:
df_raw['age'].fillna(df_raw['age'].mode()[0], inplace=True)

In [13]:
df_raw

,id,age,salary,joined,active,score
0,1,25,1000,2020-01-15,yes,85.5
1,2,thirty,"$2,500",15/02/2021,no,90
2,3,40,3000.0,2021-03-01,1,—
3,4,25,NaN,unknown,0,70.0
4,5,25,0,NaN,True,NaN
5,6,35,"$1,200",2020-12-31,FALSE,88


In [14]:
df_raw['age'].replace({'thirty': 30, 'NaN': None, '': None}, inplace=True)

In [15]:
df_raw

,id,age,salary,joined,active,score
0,1,25,1000,2020-01-15,yes,85.5
1,2,30,"$2,500",15/02/2021,no,90
2,3,40,3000.0,2021-03-01,1,—
3,4,25,NaN,unknown,0,70.0
4,5,25,0,NaN,True,NaN
5,6,35,"$1,200",2020-12-31,FALSE,88


In [17]:
# safely convert to pandas nullable integer: coerce non-numeric values to NA first
df_raw['age'] = pd.to_numeric(df_raw['age'], errors='coerce').astype('Int64')

In [18]:
df_raw['age']

0    25
1    30
2    40
3    25
4    25
5    35
Name: age, dtype: Int64

In [19]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6 non-null      int64 
 1   age     6 non-null      Int64 
 2   salary  5 non-null      object
 3   joined  5 non-null      object
 4   active  6 non-null      object
 5   score   5 non-null      object
dtypes: Int64(1), int64(1), object(4)
memory usage: 426.0+ bytes


In [ ]:
# quick cleaning and dtype conversions to practice replacing values and changing dtypes
df_clean = df_raw.replace({'N/A': pd.NA, 'NaN': pd.NA, '': pd.NA, '—': pd.NA, 'unknown': pd.NA})

# fix known word -> numeric mapping and convert
df_clean['age'] = df_raw['age'].fillna(df_raw['age'].mode()[0], inplace=True)
df_clean['age'] = df_clean['age'].replace({'thirty': '30'})
df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce').astype('Int64')  # nullable integer

# normalize salary (remove currency symbols / commas) and convert to numeric
df_clean['salary'] = df_clean['salary'].astype(str).str.replace(r'[^0-9.\-]', '', regex=True)
df_clean['salary'] = pd.to_numeric(df_clean['salary'], errors='coerce')

# parse dates
df_clean['joined'] = pd.to_datetime(df_clean['joined'], dayfirst=False, errors='coerce')

# normalize boolean-like values to pandas nullable boolean
df_clean['active'] = df_clean['active'].astype(str).str.strip().str.lower().map(
	{'yes': True, 'true': True, '1': True, 'no': False, 'false': False, '0': False}
).astype('boolean')

# numeric score
df_clean['score'] = pd.to_numeric(df_clean['score'], errors='coerce')
df_clean['score'] = df_clean['score'].fillna(df_clean['score'].mean())

print("\nCleaned data:")
print(df_clean)
print("\nDtypes (after cleaning):")
print(df_clean.dtypes)
print("\nMissing values per column:")
print(df_clean.isna().sum())


Cleaned data:
   id  age  salary     joined  active  score
0   1   25  1000.0 2020-01-15    True   85.5
1   2   30  2500.0        NaT   False   90.0
2   3   40  3000.0 2021-03-01    True    NaN
3   4   25     NaN        NaT   False   70.0
4   5   25     0.0        NaT    True    NaN
5   6   35  1200.0 2020-12-31   False   88.0

Dtypes (after cleaning):
id                 int64
age                Int64
salary           float64
joined    datetime64[ns]
active           boolean
score            float64
dtype: object

Missing values per column:
id        0
age       0
salary    1
joined    3
active    0
score     2
dtype: int64
